# Korduvad närvivõrgud

Eelmises moodulis käsitlesime teksti rikkalikke semantilisi esitusi. Kasutatud arhitektuur haarab lause sõnade koondatud tähenduse, kuid ei arvesta sõnade **järjekorda**, kuna koondamisoperatsioon, mis järgneb sisendite teisendamisele, eemaldab selle teabe algsest tekstist. Kuna need mudelid ei suuda esitada sõnade järjestust, ei saa nad lahendada keerukamaid või mitmetähenduslikke ülesandeid, nagu teksti genereerimine või küsimustele vastamine.

Tekstijada tähenduse tabamiseks kasutame närvivõrgu arhitektuuri, mida nimetatakse **korduvaks närvivõrguks** ehk RNN-iks. RNN-i kasutamisel edastame oma lause võrgu kaudu ühe tokeni korraga, ja võrk toodab mingi **seisundi**, mille edastame võrku uuesti koos järgmise tokeniga.

![Pilt, mis näitab korduva närvivõrgu genereerimise näidet.](../../../../../translated_images/et/rnn.27f5c29c53d727b5.webp)

Arvestades sisendjada tokenitest $X_0,\dots,X_n$, loob RNN närvivõrgu plokkide jada ja treenib seda jada otsast lõpuni tagasileviku abil. Iga võrguplokk võtab sisendiks paari $(X_i,S_i)$ ja annab tulemuseks $S_{i+1}$. Lõplik seisund $S_n$ või väljund $Y_n$ suunatakse lineaarse klassifikaatori kaudu tulemuse saamiseks. Kõik võrguplokid jagavad samu kaale ja neid treenitakse otsast lõpuni ühe tagasileviku käigu abil.

> Ülaltoodud joonis näitab korduvat närvivõrku lahtirullitud kujul (vasakul) ja kompaktsema korduva esitusena (paremal). Oluline on mõista, et kõik RNN-rakud jagavad samu **kaale**.

Kuna seisundivektorid $S_0,\dots,S_n$ edastatakse võrgu kaudu, suudab RNN õppida sõnade järjestuslikke sõltuvusi. Näiteks, kui sõna *mitte* ilmub kuskil jadas, suudab see õppida teatud elementide eitamist seisundivektoris.

Iga RNN-raku sees on kaks kaalumatriitsi: $W_H$ ja $W_I$, ning nihe $b$. Iga RNN-sammu puhul arvutatakse sisendi $X_i$ ja sisendseisundi $S_i$ põhjal väljundseisund järgmiselt: $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, kus $f$ on aktivatsioonifunktsioon (sageli $\tanh$).

> Probleemide puhul, nagu teksti genereerimine (mida käsitleme järgmises üksuses) või masintõlge, soovime saada väljundväärtust igal RNN-sammul. Sellisel juhul on olemas ka teine maatriks $W_O$, ja väljund arvutatakse järgmiselt: $Y_i=f(W_O\times S_i+b_O)$.

Vaatame, kuidas korduvad närvivõrgud aitavad meil klassifitseerida meie uudiste andmekogumit.

> Liivakasti keskkonna jaoks peame käivitama järgmise lahtri, et veenduda vajaliku teegi installimises ja andmete eellaadimises. Kui töötate lokaalselt, võite järgmise lahtri vahele jätta.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Suuremate mudelite treenimisel võib GPU mälu jaotamine muutuda probleemiks. Samuti võib olla vajalik katsetada erinevate minibatch'i suurustega, et andmed mahuksid GPU mällu, kuid treenimine oleks siiski piisavalt kiire. Kui käitate seda koodi oma GPU masinas, võite katsetada minibatch'i suuruse kohandamist, et treenimist kiirendada.

> **Note**: Teatud NVidia draiverite versioonid on teadaolevalt sellised, mis ei vabasta mälu pärast mudeli treenimist. Me käitame selles märkmikus mitmeid näiteid, mis võib teatud seadistustes põhjustada mälu ammendumist, eriti kui teete oma eksperimente sama märkmiku raames. Kui mudeli treenimise alustamisel ilmnevad kummalised vead, võib olla vajalik märkmiku kerneli taaskäivitamine.


In [3]:
batch_size = 16
embed_size = 64

## Lihtne RNN klassifikaator

Lihtsa RNN-i puhul on iga korduvüksus lihtne lineaarne võrk, mis võtab sisendvektori ja olekuvektori ning toodab uue olekuvektori. Kerases saab seda esindada `SimpleRNN` kihiga.

Kuigi me saame anda RNN-kihile otse ühekuumkooditud (one-hot encoded) tokenid, ei ole see hea mõte nende kõrge dimensioonilisuse tõttu. Seetõttu kasutame sõnavektorite dimensioonide vähendamiseks esmalt sisendkihina embedding-kihte, millele järgneb RNN-kiht ja lõpuks `Dense` klassifikaator.

> **Märkus**: Juhtudel, kus dimensioonilisus ei ole nii kõrge, näiteks kui kasutatakse tähemärgitase tokeniseerimist, võib olla mõistlik anda ühekuumkooditud tokenid otse RNN-rakule.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Märkus:** Siin kasutame lihtsuse huvides treenimata sisendkihti, kuid paremate tulemuste saavutamiseks võiks kasutada eeltreenitud sisendkihti, näiteks Word2Vec-i, nagu kirjeldatud eelmises osas. Hea harjutus oleks kohandada seda koodi töötama eeltreenitud sisenditega.

Nüüd treenime oma RNN-i. Üldiselt on RNN-e üsna keeruline treenida, sest kui RNN-i rakud lahti rullitakse mööda järjestuse pikkust, on tagasipropagatsioonis osalevate kihtide arv üsna suur. Seetõttu peame valima väiksema õppemäära ja treenima võrku suurema andmekogumi peal, et saavutada häid tulemusi. See võib võtta üsna kaua aega, seega on eelistatud kasutada GPU-d.

Kiiruse huvides treenime RNN-mudelit ainult uudiste pealkirjade põhjal, jättes kirjelduse välja. Võite proovida treenida koos kirjeldusega ja vaadata, kas mudel suudab treenida.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Märkus**: täpsus on siin tõenäoliselt madalam, kuna treenime ainult uudiste pealkirjade põhjal.


## Muutuvate järjestuste uuesti vaatamine

Pea meeles, et `TextVectorization` kiht lisab automaatselt pad-tokeneid muutuvate pikkustega järjestustele minibatchis. Selgub, et need tokenid osalevad samuti treeningus ja võivad mudeli koondumist keerulisemaks muuta.

On mitmeid lähenemisi, mida saame kasutada, et vähendada pad-tokeneid. Üks neist on andmekogumi ümberjärjestamine järjestuse pikkuse järgi ja kõigi järjestuste rühmitamine suuruse alusel. Seda saab teha funktsiooni `tf.data.experimental.bucket_by_sequence_length` abil (vaata [dokumentatsiooni](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Teine lähenemine on kasutada **maskimist**. Kerases toetavad mõned kihid täiendavat sisendit, mis näitab, milliseid tokeneid tuleks treeningu ajal arvesse võtta. Maskimise kaasamiseks oma mudelisse saame lisada eraldi `Masking` kihi ([dokumentatsioon](https://keras.io/api/layers/core_layers/masking/)) või määrata `Embedding` kihi parameetri `mask_zero=True`.

> **Note**: Selle treeningu läbiviimine kogu andmekogumi ühe epohhi jaoks võtab umbes 5 minutit. Kui kaotate kannatuse, võite treeningu igal ajal katkestada. Samuti saate piirata treeninguks kasutatava andmekogumi mahtu, lisades `.take(...)` klausli pärast `ds_train` ja `ds_test` andmekogumeid.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Nüüd, kui kasutame maskeerimist, saame mudelit treenida kogu pealkirjade ja kirjelduste andmestiku põhjal.

> **Märkus**: Kas oled märganud, et oleme kasutanud vektoreerijat, mis on treenitud uudiste pealkirjade, mitte kogu artikli sisu põhjal? See võib potentsiaalselt põhjustada mõnede tokenite ignoreerimist, seega oleks parem vektoreerija uuesti treenida. Kuid selle mõju võib olla väga väike, nii et lihtsuse huvides jääme eelmise eeltreenitud vektoreerija juurde.


## LSTM: Pikaajaline lühimälu

Üks peamisi RNN-ide probleeme on **kaduvad gradiendid**. RNN-id võivad olla üsna pikad ja neil võib olla raskusi gradientide tagasi esimese kihini edastamisega tagasilevi ajal. Kui see juhtub, ei suuda võrk õppida kaugemate tokenite vahelisi seoseid. Üks viis selle probleemi vältimiseks on **eksplitsiitne oleku haldamine** **väravate** abil. Kaks kõige levinumat arhitektuuri, mis kasutavad väravaid, on **pikaajaline lühimälu** (LSTM) ja **gated relay unit** (GRU). Siin käsitleme LSTM-e.

![Pilt, mis näitab pikaajalise lühimälu raku näidet](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-võrk on organiseeritud sarnaselt RNN-ile, kuid kihilt kihile edastatakse kaks olekut: tegelik olek $c$ ja peidetud vektor $h$. Igas üksuses kombineeritakse peidetud vektor $h_{t-1}$ sisendiga $x_t$, ja koos kontrollivad nad, mis juhtub olekuga $c_t$ ja väljundiga $h_{t}$ **väravate** kaudu. Igal väraval on sigmoidne aktivatsioon (väljund vahemikus $[0,1]$), mida võib mõelda kui bittmaski, kui see korrutatakse oleku vektoriga. LSTM-idel on järgmised väravad (vasakult paremale ülaloleval pildil):
* **unustamisvärav**, mis määrab, millised komponendid vektorist $c_{t-1}$ tuleb unustada ja millised edasi anda.
* **sisendvärav**, mis määrab, kui palju informatsiooni sisendvektorist ja eelmisest peidetud vektorist tuleks olekuvektorisse lisada.
* **väljundvärav**, mis võtab uue olekuvektori ja otsustab, milliseid selle komponente kasutatakse uue peidetud vektori $h_t$ loomiseks.

Olekukomponente $c$ võib mõelda kui lippe, mida saab sisse ja välja lülitada. Näiteks, kui kohtame järjestuses nime *Alice*, arvame, et see viitab naisele, ja tõstame olekus lipu, mis ütleb, et lauses on naissoost nimisõna. Kui kohtame hiljem sõnu *ja Tom*, tõstame lipu, mis ütleb, et lauses on mitmuse nimisõna. Seega, manipuleerides olekuga, saame jälgida lause grammatilisi omadusi.

> **Note**: Siin on suurepärane ressurss LSTM-ide sisemuse mõistmiseks: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) autorilt Christopher Olah.

Kuigi LSTM-raku sisemine struktuur võib tunduda keeruline, peidab Keras selle implementatsiooni `LSTM` kihi sisse, nii et ainus asi, mida me peame ülaltoodud näites tegema, on asendada korduv kiht:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


> **Märkus**: LSTM-ide treenimine on samuti üsna aeglane ja treeningu alguses ei pruugi täpsus märkimisväärselt suureneda. Hea täpsuse saavutamiseks võib olla vajalik treenimist mõnda aega jätkata.


## Kaksuunalised ja mitmekihilised RNN-id

Meie senistes näidetes on korduvad närvivõrgud töötanud järjestuse algusest lõpuni. See tundub meile loomulik, kuna see järgib sama suunda, milles me loeme või kuulame kõnet. Kuid olukordades, kus on vaja sisendjärjestusele juhuslikult ligi pääseda, on mõistlikum käivitada korduv arvutus mõlemas suunas. RNN-e, mis võimaldavad arvutusi mõlemas suunas, nimetatakse **kaksuunalisteks** RNN-ideks ning neid saab luua, mähkides korduva kihi spetsiaalse `Bidirectional` kihi sisse.

> **Note**: `Bidirectional` kiht teeb selle sees olevast kihist kaks koopiat ja seab ühe neist koopiatest omaduse `go_backwards` väärtuseks `True`, mis paneb selle liikuma järjestuses vastassuunas.

Korduvad närvivõrgud, olgu need ühe- või kaksuunalised, püüavad kinni mustreid järjestuses ja salvestavad need olekuvektoritesse või tagastavad need väljundina. Nii nagu konvolutsioonivõrkude puhul, saame esimesele kihile lisada teise korduva kihi, et püüda kinni kõrgema taseme mustreid, mis on loodud esimese kihi poolt tuvastatud madalama taseme mustritest. See viib meid **mitmekihilise RNN-i** mõisteni, mis koosneb kahest või enamast korduvast võrgust, kus eelmise kihi väljund edastatakse järgmisele kihile sisendina.

![Pilt, mis näitab mitmekihilist pika lühimälu RNN-i](../../../../../translated_images/et/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Pilt [sellest suurepärasest postitusest](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3), autor Fernando López.*

Keras muudab nende võrkude loomise lihtsaks ülesandeks, kuna peate mudelile lihtsalt lisama rohkem korduvaid kihte. Kõigi kihtide puhul, välja arvatud viimane, peame määrama parameetri `return_sequences=True`, kuna vajame, et kiht tagastaks kõik vahepealsed olekud, mitte ainult korduva arvutuse lõppoleku.

Loome kahekihilise kaksuunalise LSTM-i meie klassifitseerimisülesande jaoks.

> **Note** see kood võtab taas üsna kaua aega, et lõpule jõuda, kuid see annab meile seni nähtud kõrgeima täpsuse. Seega võib-olla tasub oodata ja tulemust näha.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN-id muudeks ülesanneteks

Siiani oleme keskendunud RNN-ide kasutamisele tekstijadade klassifitseerimiseks. Kuid need suudavad toime tulla paljude teiste ülesannetega, nagu teksti genereerimine ja masintõlge &mdash; neid ülesandeid käsitleme järgmises üksuses.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
